In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import re
import os, json

from textwrap import dedent
from pprint import pprint

import warnings
warnings.filterwarnings("ignore")

### 문제 5-1 : 카페 메뉴 도구(Tool) 호출 체인 구현

In [ ]:
from pathlib import Path
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

data_path = Path("../data/cafe_menu_data.txt")
db_path = "../db/cafe_db"

loader = TextLoader("../data/cafe_menu_data.txt", encoding="utf-8")
documents = loader.load()

splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = splitter.split_documents(documents)

embedding = OllamaEmbeddings(model="bge-m3")  
vectorstore = FAISS.from_documents(docs, embedding)
vectorstore.save_local(db_path)

In [27]:
from langchain.tools import tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

@tool
def wiki_summary(query: str) -> str:
    """위키피디아에서 요약된 정보를 제공합니다."""
    wiki = WikipediaAPIWrapper(lang="ko")
    return wiki.run(query)

@tool
def tavily_search_func(query: str) -> str:
    """웹에서 최신 정보를 검색합니다."""
    search = TavilySearchResults(k=3)
    return search.run(query)

@tool
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 관련 정보를 검색합니다."""
    db = FAISS.load_local("../db/cafe_db", OllamaEmbeddings(model="bge-m3:latest"), allow_dangerous_deserialization=True)
    docs = db.similarity_search(query, k=2)
    return "\n".join([doc.page_content for doc in docs])


In [28]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType

llm = ChatOpenAI(model="gpt-4", temperature=0)

tools = [wiki_summary, tavily_search_func, db_search_cafe_func]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

question = "아메리카노의 가격과 특징은 무엇인가요?"
response = agent.run(question)

print("\n🔍 최종 응답:\n", response)




> Entering new AgentExecutor chain...

Invoking: `db_search_cafe_func` with `{'query': '아메리카노'}`


1. 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 뜨거운 물
   • 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.
9. 아이스 아메리카노
   • 가격: ₩4,500
   • 주요 원료: 에스프레소, 차가운 물, 얼음
   • 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.아메리카노는 다음과 같은 특징과 가격을 가지고 있습니다.

1. 아메리카노
   - 가격: ₩4,500
   - 주요 원료: 에스프레소, 뜨거운 물
   - 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가능합니다.

2. 아이스 아메리카노
   - 가격: ₩4,500
   - 주요 원료: 에스프레소, 차가운 물, 얼음
   - 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.

> Finished chain.

🔍 최종 응답:
 아메리카노는 다음과 같은 특징과 가격을 가지고 있습니다.

1. 아메리카노
   - 가격: ₩4,500
   - 주요 원료: 에스프레소, 뜨거운 물
   - 설명: 진한 에스프레소에 뜨거운 물을 더해 만든 클래식한 블랙 커피입니다. 원두 본연의 맛을 가장 잘 느낄 수 있으며, 깔끔하고 깊은 풍미가 특징입니다. 설탕이나 시럽 추가 가

### 문제 5-2 : Few-shot 프롬프팅을 활용한 카페 AI 어시스턴트

In [29]:
from datetime import datetime
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_openai import ChatOpenAI

examples = [
    HumanMessage("아메리카노 정보와 커피 역사를 알려주세요", name="example_user"),
    AIMessage("메뉴 검색과 위키피디아 검색을 진행하겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[
        {"name": "db_search_cafe_func", "args": {"query": "아메리카노"}, "id": "1"}
    ]),
    ToolMessage("아메리카노: ₩4,500, 에스프레소에 뜨거운 물을 넣은 커피 음료입니다.", tool_call_id="1"),
    AIMessage("이제 커피의 역사 정보를 찾아보겠습니다.", name="example_assistant"),
    AIMessage("", name="example_assistant", tool_calls=[
        {"name": "wiki_summary", "args": {"query": "커피 역사"}, "id": "2"}
    ]),
    ToolMessage("커피는 에티오피아에서 유래되었으며, 이후 중동과 유럽을 거쳐 전 세계적으로 확산되었습니다.", tool_call_id="2"),
    AIMessage("아메리카노(₩4,500)는 에스프레소에 뜨거운 물을 넣은 커피 음료입니다. 커피는 에티오피아에서 유래되어 전 세계로 퍼졌습니다.", name="example_assistant")
]

system = """당신은 카페 메뉴 정보와 일반적인 음식/음료 지식을 제공하는 AI입니다.

도구 사용 가이드라인:
- db_search_cafe_func: 카페 메뉴 정보 (가격, 재료, 설명)
- wiki_summary: 일반 지식 (역사, 제조법, 문화적 배경)
- tavily_search_func: 최신 정보 (트렌드, 뉴스, 실시간 정보)

사용 원칙:
1. 카페 메뉴 관련 질문 → 반드시 메뉴 DB 먼저 검색
2. 역사/문화/일반 지식 → 위키피디아 활용
3. 최신 트렌드/뉴스 → 웹 검색 활용
4. 복합 질문 → 여러 도구 순차 사용
5. 정보 출처를 명확히 구분하여 답변

오늘 날짜: """ + datetime.today().strftime("%Y-%m-%d")

few_shot_prompt = ChatPromptTemplate.from_messages([
    ("system", system),
    *examples,
    ("human", "{user_input}"),
    ("placeholder", "{messages}"),
])

llm = ChatOpenAI(model="gpt-4o-mini")

tools = [db_search_cafe_func, wiki_summary, tavily_search_func] 
llm_with_tools = llm.bind_tools(tools=tools)

fewshot_search_chain = few_shot_prompt | llm_with_tools


In [31]:

@chain
def cafe_assistant_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = fewshot_search_chain.invoke(input_, config=config)

    tool_msgs = []
    for tool_call in ai_msg.tool_calls:
        print(f"{tool_call['name']} → args: {tool_call['args']}")
        if tool_call["name"] == "db_search_cafe_func":
            tool_msg = db_search_cafe_func.invoke(tool_call, config=config)
        elif tool_call["name"] == "wiki_summary":
            tool_msg = wiki_summary.invoke(tool_call, config=config)
        elif tool_call["name"] == "tavily_search_func":
            tool_msg = tavily_search_func.invoke(tool_call, config=config)
        else:
            continue
        tool_msgs.append(tool_msg)

    return fewshot_search_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)

query = "카페라떼와 어울리는 디저트는 무엇인가요? 그리고 라떼의 유래도 알려주세요."
response = cafe_assistant_chain.invoke(query)

pprint(response.content)


db_search_cafe_func → args: {'query': '카페라떼'}
wiki_summary → args: {'query': '라떼의 유래'}
('### 카페라떼 정보\n'
 '**카페라떼**는 에스프레소에 뜨거운 우유를 더한 커피 음료입니다. 일반적으로 가격은 ₩6,000입니다. 카페라떼는 부드러운 우유와 '
 '커피의 조화로 편안한 맛을 제공합니다.\n'
 '\n'
 '### 카페라떼와 어울리는 디저트\n'
 '카페라떼와 잘 어울리는 디저트로는 다음과 같은 것들이 있습니다:\n'
 '- **케이크**: 초콜릿 케이크, 치즈 케이크 등.\n'
 '- **파이**: 사과 파이, 블루베리 파이.\n'
 '- **쿠키**: 초코칩 쿠키, 오트밀 쿠키.\n'
 '\n'
 '이러한 디저트들은 카페라떼의 크리미한 맛과 잘 어울려 즐거운 조화를 이룹니다.\n'
 '\n'
 '### 라떼의 유래\n'
 '라떼(또는 카페 라떼)는 에스프레소에 데운 우유를 혼합한 음료로, “우유가 들어간 커피”를 의미합니다. 역사는 이탈리아에서 시작되었으며, '
 '전통적으로는 커피와 우유의 비율이 1:3 또는 1:2로 되어 있습니다. 라떼는 일반적으로 아침 식사와 함께 제공되며, 에스프레소와 우유 '
 '비율에 따라 크기와 농도가 달라질 수 있습니다.')
